### Unpack ETL files

In [15]:
import os
from pathlib import Path

from openai.types.beta.threads import image_file

project_root = Path().resolve()
etl_dir = project_root / "data" / "ETL8G"
unpack_script = project_root / "data" / "unpack_etlcdb" / "unpack_etlcdb" / "unpack.py"

files = os.listdir(etl_dir)

for file in files:
    if file != "ETL8INFO":
        input_file = etl_dir / file
        cmd = f'python {unpack_script} {input_file}'
        print("Running:", cmd)
        # os.system(cmd)



Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_01
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_02
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_03
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_04
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\unpack_etlcdb\unpack_etlcdb\unpack.py C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\ETL8G\ETL8G_05
Running: python C:\Users\alicj\PycharmProjects\KanjiRecognitionModel\data\u

### Saving labels to list

In [106]:
import pandas as pd

path_to_labels = Path().resolve()/ "data" / "ETL8G"/ "ETL8G_01_unpack"/"meta.csv"

labels_df = pd.read_csv(path_to_labels)

labels = labels_df["char"]
labels = labels.tolist()


### Saving images

In [51]:
image_paths = []
img_dir_path = Path().resolve()/ "data" / "ETL8G"

files = os.listdir(img_dir_path)
for file in files:
    if "unpack" in file:
        unpack_dir = img_dir_path / file
        img_files = os.listdir(unpack_dir)
        for img_file in img_files:
            if img_file != 'meta.csv':
                img_path = unpack_dir / img_file
                image_paths.append(img_path)

# in the last dir there are only 955 images



### Create a CNN

In [66]:
from torch.utils.data import Dataset, DataLoader, random_split
import torch

tensor([[1., 2.],
        [3., 4.]])
torch.Size([2, 2])


False


In [109]:
print(len(image_paths))
print(len(labels))

153916
153916


In [107]:
labels_copy = labels.copy()

for i in range(31):
    labels_copy_2 = labels_copy.copy()
    labels.extend(labels_copy_2)

labels.extend(labels_copy[:956])

In [108]:
print(len(labels))

153916


### Dataset class

In [111]:
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]



In [112]:
dataset = ImageDataset(image_paths, labels)

### Splitting the data into train, val, test sets

In [114]:
from torch.utils.data import random_split

# First split into train+val and test (80/20)
train_val_size = int(0.8 * len(dataset))
test_size      = len(dataset) - train_val_size
train_val_dataset, test_dataset = random_split(dataset, [train_val_size, test_size])

# Then split train+val into train and val (e.g. 75/25 of that 80%)
train_size = int(0.75 * len(train_val_dataset))
val_size   = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])
